In [1]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO,DQN,SAC
from stable_baselines3.common.evaluation import evaluate_policy
from scipy.special import softmax
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.logger import configure
import os
#The hand of god shall strike where the faithful need help the most. 
from gymnasium import spaces
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.logger import configure
import os
import time
from stable_baselines3.common.callbacks import BaseCallback
import torch as th
from torch import nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import ActorCriticPolicy


In [8]:
#Essential functions for the environments
def entropy(p):
    if p == 0 or p == 1:
        return 0  # Standard entropy definition
    return -p * np.log2(p) - (1 - p) * np.log2(1 - p)
def confidence(L, vote):
    TP, TN, FN, FP = L
    
    if vote == 1:
        # Considering only positive votes
        total_positives = TP + FP
        if total_positives == 0:
            return 0.0  # No positive predictions to consider
        p_TP_given_positive = TP / total_positives
        # Return 1 minus entropy to inverse the effect for your use case
        return 1 - entropy(p_TP_given_positive)  # This reflects 'confidence' as per your definition

    elif vote == -1:
        # Considering only negative votes
        total_negatives = TN + FN
        if total_negatives == 0:
            return 0.0  # No negative predictions to consider
        p_TN_given_negative = TN / total_negatives
        # Return 1 minus entropy to inverse the effect for your use case
        return 1 - entropy(p_TN_given_negative)
def ent(plus,negative):
    #entropy of dataset
    total= plus+negative
    if total == 0.:
        return(0)
    p= plus/total
    q= negative/total
    if p == 0:
        first =0.
    else:
        first= -p*np.log2(p)
    if q==0:
        second =0.
    else:
       second= - q*np.log2(q)
    return(first+second)

def split(TP,TN,FN,FP,func= ent):
    # it considers the classifer (expert) which has performance characteristics and returns the ent of the corresponding split
    if TP+TN+FN+FP == 0:
        return(1.)
    t= (TP+FP)/(TP+FP+FN+TN)
    u= (FN+TN)/(TP+FP+FN+TN)
    return (t* func(TP,FP)+u* func(FN,TN))

def drop(TP,FP,FN,TN,func= ent, P=0.5):
    return(func(TP+FN,FP+TN)-split(TP,TN,FN,FP,func))


def split2(L,func = ent):
    return(split(L[0],L[1],L[2],L[3],func=func))



def drop2(L, func=ent, p=0.5):
    #same with split L should be [TP,TN,FN,FP]
    return(drop(L[0],L[1],L[2],L[3],func=func,P=p))
def conf1(L,real,vote,p=0.5):
    # L should be [TP,TN,FN,FP]
    TP=L[0]
    TN=L[1]
    FN=L[2]
    FP=L[3]

    P= p
    N= 1- P
    match real,vote:
        case 1,1:
            if TP+FP==0:
                return(P)
            else:
                return(TP/(TP+FP))
        case 1,-1:
            if TN+FN==0:
                return(N)
            else:
                return(FN/(TN+FN))
        case -1,-1:
            if TN+FN==0:
                return(N)
            else:
                return(TN/(TN+FN))
        case -1,1:
            if TP+FP==0:
                return(P)
            else:
                return(FP/(TP+FP))

def caster(L, vote, mode=None):
    TP, TN, FN, FP = L
    #BE VERY CAREFUL PLEASE!!!!!!!!!!  THE DEFINITIONS OF ENTROPY AND CONFUSION MATRIX HAVE BEEN REVERSED, DONE SO FOR CODE SIMPLICITY BUT MUST BE CAREFUL
    if mode == None:
        if vote == -1 and TN == 0 and FN == 0:
            return -1.0  # Explicitly handle the edge case for negative vote
        if vote == 1 and TP == 0 and FP == 0:
            return 1.0  # Explicitly handle the edge case for positive vote
        cast = 1 * conf1(L, 1, vote) - 1 * conf1(L, -1, vote)
        return cast
    
    if mode == "entropy":
        if vote == -1 and TN == 0 and FN == 0:
            return -1.0  # Explicitly handle the edge case for negative vote
        if vote == 1 and TP == 0 and FP == 0:
            return 1.0  # Explicitly handle the edge case for positive vote
        vote_multiplier = np.sign((vote + 1) * (L[0] - L[3]) / 2 + (vote - 1) * (L[1] - L[2]) / 2)
        return vote_multiplier * confidence(L=L, vote=vote)
    
def guess(val,mu,var):
    limit= val+ np.random.uniform(-(3**(1/2))*var,(3**(1/2))*var)
    if limit>= -mu:
        return(1.0)
    return(-1.0)

def transformer(dat,logaritmic_observation=False,n=800):
    # data 0=number_of_times_played 1=TP, 2=FP, 3=FN, 4=TN ( modulo 5 )

    #--------> NEW 1=TP, 2=TN, 3=FN, 4=FP
    #take in data and return observation
    #
    this = np.array(dat, dtype=float)
    if this[0] == 0.:
        if logaritmic_observation == False:
            return(np.array([0.,0.5,0.5,0.,0.]))
        if logaritmic_observation == True:
            return(np.array([np.log(n)+1,0.5,0.5,0.,0.]))
    
    ret = np.zeros_like(this)
    ret[0] = this[0]
    if logaritmic_observation == True:
        ret[0] = np.log(n/this[0])
    for i in range(4):
        ret[i+1]=this[1+i]/this[0]
    return(ret)
def stochastic_round(x):
    if x == int(x):
        return int(x)
    
    lower = int(np.floor(x))
    upper = int(np.ceil(x))
    
    prob_upper = x - lower
    
    return np.random.choice([lower, upper], p=[1 - prob_upper, prob_upper])


class NormalizeObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        # Define the bounds for normalization based on the observation space of Expert1
        self.low = self.observation_space.low
        self.high = self.observation_space.high

    def observation(self, observation):
        # Normalize the observation to range [0, 1]
        normalized_obs = (observation - self.low) / (self.high - self.low)
        return normalized_obs

def linear_map(value, low, high):

    # Ensure inputs are numpy arrays
    value = np.asarray(value)
    low = np.asarray(low)
    high = np.asarray(high)
    
    # Calculate the denominator, and set it to 1 where low == high to avoid division by zero
    denominator = np.where(low != high, high - low, 1)
    
    # Perform the linear mapping
    transformed_value = 2 * (value - low) / denominator - 1
    
    # Set transformed values to 0 where low == high
    transformed_value[low == high] = 0
    
    return np.float32(transformed_value)



In [69]:
class Expert3(gym.Env):
    def __init__(s, n=3,turns=8, order_type="no_ordering",cost_range=[0.,0.125], perform_evaluation=True,normalized=False,logaritmic_obs= True):
        s.n = n  # number of experts
        s.turns = turns  # number of patients
        s.round = 0  # the expert we choose to consult or not rn depending on ordering
        s.t = 0  # the patient that we're treating rn
        s.logaritmic_obs=logaritmic_obs
        s.order_mode = order_type  # environment ordering type
        s.perform_evaluation = perform_evaluation  # Add perform_evaluation boolean
        s.normalized= normalized
        s.cost_range=cost_range
        nbr = gym.spaces.Box(low=0., high=s.turns, shape=[s.n,])  # number of visits to each expert
        P_trust = gym.spaces.Box(low=-1., high=1., shape=(s.n,))  # how much we trust the Positive vote of an expert
        N_trust = gym.spaces.Box(low=-1., high=1., shape=(s.n,))  # how much we trust the Negative vote of an expert
        # The observations go [nbr of visits, P_trust, N_trust,P rate, cost, taken]
        s.low = np.array([0, -1, -1, 0, s.cost_range[0],0])
        s.high = np.array([s.turns-1, 1, 1, 1, s.cost_range[1],2])
        if logaritmic_obs == True:
            s.high= np.array([np.log(s.turns)+1, 1, 1, 1, s.cost_range[1],2])

        taken = gym.spaces.MultiDiscrete(np.array([3] * s.n))
        s.loss_for_softmax = np.array([0.] * s.n)
        other = gym.spaces.Box(low=np.array([0., 0., 0., 0.]), high=np.array([s.turns - 1, s.turns - 1, s.n, s.n]), shape=(4,))
        active_round = gym.spaces.MultiBinary(1)


        #s.observation_space = gym.spaces.Dict({"nbr": nbr, "P_trust": P_trust, "N_trust": N_trust, "taken": taken, "other": other, "active_round": active_round})


        s.low_matrix = np.tile(s.low, (s.n, 1))
        s.high_matrix = np.tile(s.high, (s.n, 1))

        s.observation_space=gym.spaces.Dict({"mat":gym.spaces.Box(low=s.low_matrix, high=s.high_matrix,dtype=np.float32), "other":gym.spaces.Box(low=0,high=s.turns-1)})
        if normalized== True:
            s.observation_space = spaces.Dict({
            "mat": spaces.Box(low=-1*np.ones_like(s.low_matrix), high=np.ones_like(s.high_matrix), dtype=np.float32),
            "other": spaces.Box(low=np.array([-1]), high=np.array([1]), dtype=np.float32)
        })

        

        s.action_space = gym.spaces.MultiBinary(1)
        s.mu_window = 1.0
        s.var_window = 1.0
        s.cost = 0.125
        if s.perform_evaluation:
            s.rewards_list = []  # Initialize the rewards list
            s.experts_consulted = []  # Initialize the experts consulted list
            s.correct_classifications = []  # Initialize the correct classifications list
            s.experts_visited = []  # Initialize the experts visited list

    def reset(s, seed=0):
        s.exp_mu = np.random.uniform(-s.mu_window, s.mu_window, s.n)  # bias of experts
        s.exp_var = np.random.uniform(0.0, s.var_window, s.n)  # variance of experts
        s.costs= np.random.uniform(s.cost_range[0],s.cost_range[1],s.n)
        s.l = np.random.uniform(-1.0, 1.0, s.turns)  # the patients
        s.data = np.float32(np.array([0., 0., 0., 0., 0.] * s.n))
        s.nbr = np.array([0.] * s.n)  # number of visits to each expert
        s.P_trust = np.float32(np.array([0.] * s.n))  # p trust vector
        s.N_trust = np.float32(np.array([0.] * s.n))  # n trust vector

        
        s.p_rate=np.array([0.]*s.n,dtype=np.float32)

        
        s.taken = np.array([2] * s.n, dtype=int)  # taken or not or not yet vector
        s.other = np.float32(np.array([0, s.turns - 1, 0, s.n]))  # other relevant data
        s.round_rews = np.zeros(s.n)  # rewards for that patient
        s.diagnoses = np.zeros(shape=(s.n, s.turns), dtype=np.int8)  # expert*patient diagnosis matrix
        s.ground_truth = np.zeros_like(s.l)  # patient health ground truth
        s.final = np.zeros_like(s.l)  # final classification for patient t
        s.res = np.zeros_like(s.l)
        s.giver = np.float32(np.array([0.0, 0.5, 0.5, 0., 0.0] * s.n))  # for observation?
        s.loss_for_softmax = np.array([0.] * s.n)  # loss_used only for the softmax algorithm
        if s.perform_evaluation:
            s.rewards_list = []  # Reset the rewards list at the beginning of an episode
            s.experts_consulted = []  # Reset the experts consulted list at the beginning of an episode
            s.correct_classifications = []  # Reset the correct classifications list at the beginning of an episode
            s.experts_visited = []  # Reset the experts visited list at the beginning of an episode
        for i in range(s.n):
            s.nbr[i] = s.giver[5 * i]
            s.P_trust[i] = caster(transformer(s.data[i * 5:i * 5 + 5])[1:], 1)
            s.N_trust[i] = caster(transformer(s.data[i * 5:i * 5 + 5])[1:], -1)
            s.p_rate=np.array(transformer(s.data[i * 5:i * 5 + 5])[1]+transformer(s.data[i * 5:i * 5 + 5])[4])
        for i in range(s.turns):
            s.ground_truth[i] = guess(s.l[i], 0.0, 0.0)
            for j in range(s.n):
                s.diagnoses[j][i] = guess(s.l[i], s.exp_mu[j], s.exp_var[j])
        s.round = 0
        s.t = 0
        obs, info = s.get_obs(), s.get_info()
        return obs, info

    def step(s, action):
        terminated, truncated = False, False
        if s.round != s.n:  # within round expert selection
            #s.taken[s.round] = action
            s.taken[s.round] = action.item()
            extra_points = (1 / s.n) * (action - 0.5) * 2 if s.ground_truth[s.t] == s.diagnoses[s.round][s.t] else -(1 / s.n) * (action - 0.5) * 2
            s.loss_for_softmax[s.round] = 1. if s.ground_truth[s.t] == s.diagnoses[s.round][s.t] else -1.0
            rew = [-action * s.costs[s.round] + extra_points][0]
            #s.round_rews[s.round] = extra_points


            s.round_rews[s.round] = extra_points.item()
            s.round += 1
            obs = s.get_obs()
            info = s.get_info()  # Get info at each step
  # Append the reward to the rewards list if perform_evaluation is True
        else:
            num_experts_consulted = (s.taken == 1).sum()
            experts_visited = np.where(s.taken == 1)[0].tolist()
            for i in range(s.n):
                if s.taken[i] == 1:
                    s.data[i * 5] += 1  # number of times used is stored on 0th feature of observation data modulo 5
                    if s.diagnoses[i][s.t] == s.ground_truth[s.t]:
                        if s.ground_truth[s.t] == 1.0:
                            s.data[i * 5 + 1] += 1
                        elif s.ground_truth[s.t] == -1.0:
                            s.data[i * 5 + 2] += 1
                    else:
                        if s.ground_truth[s.t] == 1.0:
                            s.data[i * 5 + 3] += 1
                        elif s.ground_truth[s.t] == -1.0:
                            s.data[i * 5 + 4] += 1
            for i in range(s.n):
                s.giver[i * 5:i * 5 + 5] = transformer(s.data[i * 5:i * 5 + 5])
            votes = np.zeros(s.n)
            s.loss_for_softmax = np.array([0.] * s.n)
            for i in range(s.n):
                votes[i] = caster(transformer(s.data[i * 5:i * 5 + 5])[1:], s.diagnoses[i][s.t])
            votes = votes * s.taken
            s.final[s.t] = guess(votes.mean(), 0., 0.)
            for i in range(s.n):
                s.nbr[i] = s.giver[5 * i]
                s.P_trust[i] = caster(transformer(s.data[i * 5:i * 5 + 5])[1:], 1)
                s.N_trust[i] = caster(transformer(s.data[i * 5:i * 5 + 5])[1:], -1)
                s.p_rate=np.array(transformer(s.data[i * 5:i * 5 + 5])[1]+transformer(s.data[i * 5:i * 5 + 5])[4])

            s.round = 0
            rew = s.final[s.t] * s.ground_truth[s.t] - s.round_rews.sum() - (s.taken*s.costs).sum()
            correct_classification = s.final[s.t] == s.ground_truth[s.t]

            if s.perform_evaluation:
                s.rewards_list.append(s.final[s.t] * s.ground_truth[s.t] - np.sum(s.costs[experts_visited]))  # Append the reward to the rewards list if perform_evaluation is True
                s.experts_consulted.append(num_experts_consulted)  # Append the number of experts consulted
                s.correct_classifications.append(correct_classification)  # Append the correctness of the classification
                s.experts_visited.append(experts_visited)  # Append the list of experts visited
            s.round_rews = np.zeros(s.n)
            s.t += 1
            s.taken = np.array([2] * s.n, dtype=int)
            obs = s.get_obs()

            if s.t >= s.turns:
                truncated, terminated = True, True
            info = s.get_info()  # Get info at the end of each patient consultation
        #return obs, float(rew), terminated, truncated, info
        return obs, float(rew.item()), terminated, truncated, info

    def get_obs(s):
        active_round = np.int8(np.array([1 if s.round != s.n else 0]))
        permut = np.int8(s.get_ordering())
        other = np.float32(np.array([s.t, s.turns - s.t - 1, s.round, s.n - s.round]))

        obs1 = np.zeros_like(s.low_matrix)
        for i in range(s.n):
            obs1[i][0] = np.array(s.giver[5 * i])
            if s.logaritmic_obs == True:
                obs1[i][0] = np.array(transformer(s.data[i * 5:i * 5 + 5],n=s.turns,logaritmic_observation=s.logaritmic_obs)[0])
            obs1[i][1] = np.array(caster(transformer(s.data[i * 5:i * 5 + 5])[1:], 1))
            obs1[i][2] = np.array(caster(transformer(s.data[i * 5:i * 5 + 5])[1:], -1))
            obs1[i][3] = np.array(transformer(s.data[i * 5:i * 5 + 5])[1]+transformer(s.data[i * 5:i * 5 + 5])[4])
            obs1[i][4] = s.costs[i]

            obs1[i][5] = s.taken[i]
    
        obs1=obs1[permut]
        obs1= np.float32(obs1)
        obs2= np.array([s.t],dtype=np.float32)


        obs = {"mat": obs1, "other": obs2}
        if s.normalized== True:
            obs= {"mat": linear_map(value=obs1,low=s.low_matrix,high=s.high_matrix), "other": linear_map(value=obs2,low=0,high= s.turns-1)}
            #print(obs)
        return obs

    def get_ordering(s, forced_order=None):
        order = forced_order
        if forced_order is None:
            order = s.order_mode
        unordered = np.float32(np.arange(s.n))
        if s.round == s.n:
            return unordered
        match order:
            case "no_ordering":
                unordered[s.round] = -np.inf
                permutation = np.argsort(unordered)
                return permutation
            case "greedy":
                metric = -s.P_trust + s.N_trust
                metric[s.round] = -np.inf
                permutation = np.argsort(metric)
                return permutation

    def get_info(s):
        if s.perform_evaluation:
            return {
                "rewards": s.rewards_list,
                "experts_consulted": s.experts_consulted,
                "correct_classifications": s.correct_classifications,
                "experts_visited": s.experts_visited,
                "exp_mu": s.exp_mu.tolist(),
                "exp_var": s.exp_var.tolist(),
                "exp_costs": s.costs.tolist()
            }
        return {}

env3 = Expert3()
#env4 = Expert3(order_type="greedy")
check_env(env=env3, warn=True)
#check_env(env=env4, warn=True)
env3.reset()
#env4.reset()
nenv3=Expert3( normalized=True)
check_env(env=nenv3, warn=True)
nenv3.reset()



def play_episode(env, agent):
    obs, info = env.reset()
    done, truncated = False, False
    while not (done or truncated):
        action, _states = agent.predict(obs, deterministic=True)
        obs, reward, truncated, done, info = env.step(action)
    return info

# Initialize environment and agent


d:\Users\arman\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\env_checker.py:244: UserWarning: Your observation mat has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


In [4]:

class CustomFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Dict):
        mat_shape = observation_space.spaces['mat'].shape
        other_shape = observation_space.spaces['other'].shape
        mat_rows, mat_cols = mat_shape
        num_features_per_row = 2
        total_features_for_mat = mat_rows * num_features_per_row
        features_dim = total_features_for_mat + other_shape[0]

        super().__init__(observation_space, features_dim=features_dim)
        self.mat_extractor = nn.Sequential(
            nn.Conv2d(1, num_features_per_row, kernel_size=(1, mat_cols), stride=1),
            nn.LeakyReLU(),
            nn.Flatten()
        )

    def forward(self, observations):
        mat = observations['mat'].float().unsqueeze(1)
        mat_features = self.mat_extractor(mat)
        other_features = observations['other'].float()
        combined_features = th.cat([mat_features, other_features], dim=1)
        return combined_features
def save_model(model, filename):
    # Prepare hyperparameters, excluding non-serializable 'clip_range'
    hyperparameters = {
        'learning_rate': model.learning_rate,
        'n_steps': model.n_steps,
        'batch_size': model.batch_size,
        'n_epochs': model.n_epochs,
        'gamma': model.gamma,
        'gae_lambda': model.gae_lambda,
        'clip_range_vf': model.clip_range_vf,
        'normalize_advantage': model.normalize_advantage,
        'ent_coef': model.ent_coef,
        'vf_coef': model.vf_coef,
        'max_grad_norm': model.max_grad_norm,
        'use_sde': model.use_sde,
        'sde_sample_freq': model.sde_sample_freq,
        'target_kl': model.target_kl,
        'seed': model.seed
    }

    model_data = {
        'state_dict': model.policy.state_dict(),
        'hyperparameters': hyperparameters,
        'policy_kwargs': model.policy_kwargs
    }

    # Save the model data to a file
    th.save(model_data, filename)
def load_model(filename, env, device='cpu'):
    model_data = th.load(filename, map_location=device)

    # Set a default value for clip_range if it's critical for reinitialization
    default_clip_range = 0.2  # Default value or calculate based on other conditions

    # Reinitialize the model
    loaded_model = PPO(
        "MultiInputPolicy",
        env,
        verbose=1,
        learning_rate=model_data['hyperparameters']['learning_rate'],
        n_steps=model_data['hyperparameters']['n_steps'],
        batch_size=model_data['hyperparameters']['batch_size'],
        n_epochs=model_data['hyperparameters']['n_epochs'],
        gamma=model_data['hyperparameters']['gamma'],
        gae_lambda=model_data['hyperparameters']['gae_lambda'],
        clip_range=default_clip_range,  # Use the default or recalculated value
        clip_range_vf=model_data['hyperparameters']['clip_range_vf'],
        normalize_advantage=model_data['hyperparameters']['normalize_advantage'],
        ent_coef=model_data['hyperparameters']['ent_coef'],
        vf_coef=model_data['hyperparameters']['vf_coef'],
        max_grad_norm=model_data['hyperparameters']['max_grad_norm'],
        use_sde=model_data['hyperparameters']['use_sde'],
        sde_sample_freq=model_data['hyperparameters']['sde_sample_freq'],
        target_kl=model_data['hyperparameters']['target_kl'],
        policy_kwargs=model_data['policy_kwargs'],
        seed=model_data['hyperparameters']['seed'],
        device=device
    )
    
    # Load the state dictionary
    loaded_model.policy.load_state_dict(model_data['state_dict'])
    loaded_model.policy.eval()

    return loaded_model
model = PPO(
    "MultiInputPolicy",
    nenv3,
    verbose=1,
    policy_kwargs={
        'features_extractor_class': CustomFeatureExtractor,
        'features_extractor_kwargs': {},
        'net_arch': [24, 24]  # Modifying the network architecture from the default [64, 64]
    }
)

# Sample an observation directly from the observation space
a = nenv3.observation_space.sample()
print("Sampled observation:", a)

# Predict with the original model
print("Using original model:")
original_output, _ = model.predict(a, deterministic=True)
print("Output before saving:", original_output)

# Save the model
save_model(model, 'model.pth')

# Delete the model to simulate a fresh environment
del model

# Load the model
loaded = load_model('model.pth', env=nenv3)

# Predict with the loaded model
print("Using loaded model:")
loaded_output, _ = loaded.predict(a, deterministic=True)
print("Loaded Output after loading:", loaded_output)

# Compare outputs to check if they are the same
print("Are outputs the same?", th.equal(th.tensor(original_output), th.tensor(loaded_output)))


NameError: name 'nenv2' is not defined